In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from __future__ import print_function
import argparse
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
from torch.distributions.categorical import Categorical
from torchsummary import summary
import matplotlib.pyplot as plt
import genattack_class as GA
import time
#import efficientnet as EN

In [0]:
## Neural Network Architecture for MNIST and similar datasets
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [0]:
## Neural Network Architecture for CIFAR-10
class Net2(nn.Module):
    def __init__(self):
        super(Net2, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output

In [0]:
def adv_train(model, device, train_loader, optimizer, epoch, epsilon=0.1, alpha=0.5):
    log_interval = 120
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        #print(data.shape)
        data.requires_grad = True
        
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward(retain_graph=True)

        # Generating Adversarial Examples
        # Collect gradient of inputs
        data_grad = data.grad.data

        # FGSM Attack
        perturbed_data = fgsm_attack(data, epsilon, data_grad)

        # Get loss from perturbed data
        adv_output = model(perturbed_data)
        adv_loss = F.nll_loss(adv_output, target)


        optimizer.zero_grad()

        # Calculate total loss
        total_loss = alpha*loss + (1-alpha)*adv_loss
        total_loss.backward()

        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [0]:
def train(model, device, train_loader, optimizer, epoch, epsilon=0.1, alpha=0.5):
    log_interval = 120
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad() 
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [0]:
def fgsm_attack(image, epsilon, data_grad):
    # Collect the element-wise sign of the data gradient
    sign_data_grad = data_grad.sign()
    # Create the perturbed image by adjusting each pixel of the input image
    perturbed_image = image + epsilon*sign_data_grad
    # Adding clipping to maintain [0,1] range
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    # Return the perturbed image
    return perturbed_image

In [0]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [0]:
def adv_test(model, device, test_loader, epsilon=0.1):
    model.eval()
    test_loss = 0
    correct = 0
    adv_examples = []
    for batch_data, batch_target in test_loader:
      for i in range(batch_data.shape[0]):
        data = batch_data[i].unsqueeze(0)
        target = batch_target[i].unsqueeze(0)
        data, target = data.to(device), target.to(device)
        data.requires_grad = True
        
        output = model(data)
        
        init_pred = output.max(1, keepdim=True)[1]
        
        #print(init_pred)
        if init_pred[0].item() != target.item():
            continue

        loss = F.nll_loss(output, target)
        model.zero_grad()

        loss.backward()

        data_grad = data.grad.data

        perturbed_data = fgsm_attack(data, epsilon, data_grad)

        output = model(perturbed_data)

        final_pred = output.max(1, keepdim=True)[1]
        if final_pred.item() == target.item():
            correct += 1
        else:
            if len(adv_examples) < 5:
                adv_ex = perturbed_data.squeeze().detach().cpu().numpy()
                adv_examples.append((init_pred.item(), final_pred.item(), adv_ex))
        
    final_acc = correct/float(len(test_loader))

    print("Epsilon: {}\tTest Accuracy = {} / {} = {}\n".format(epsilon, correct, len(test_loader), final_acc))
    return final_acc, adv_examples

In [0]:
def main():
    # Training settings
    batch_size = 100
    test_batch_size = 1000
    lr = 0.1 # learning rate
    gamma = 0.7 # learning rate decay
    epochs = 20
    save_model = True

    use_cuda = torch.cuda.is_available()
    print("CUDA? " + str(use_cuda))

    torch.manual_seed(20294798)

    device = torch.device("cuda" if use_cuda else "cpu")

    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

    train_loader = torch.utils.data.DataLoader(
        datasets.CIFAR10('../data' ,train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                       ])),
        batch_size=batch_size, shuffle=True, **kwargs)
    
    test_loader = torch.utils.data.DataLoader(
        datasets.CIFAR10('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                       ])),
        batch_size=test_batch_size, shuffle=True, **kwargs)

    model = EN.EfficientNetB0().to(device)
    model_adv = EN.EfficientNetB0().to(device)

    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    optimizer_adv = optim.SGD(model_adv.parameters(), lr=lr, momentum=0.9)

    #scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
    for epoch in range(1, epochs + 1):
        print("Conventional Model")
        train(model, device, train_loader, optimizer, epoch)
        print("Adversarially trained Model")
        adv_train(model_adv, device, train_loader, optimizer_adv, epoch, epsilon = 0.1)
    print("\nTest on standard model:")
    test(model, device, test_loader)
    #print("Now with adversarial examples")
    #acc, ex = adv_test(model, device, test_loader, epsilon = 0.1)

    print("\nTest on adversarially trained model:")
    test(model_adv, device, test_loader)
    #print("Now with adversarial examples")
    #adv_acc, adv_ex = adv_test(model_adv, device, test_loader, epsilon = 0.1)

        #scheduler.step()

    if save_model:
        torch.save(model.state_dict(), "/content/drive/My Drive/NNProj/MNIST.pt")
        torch.save(model_adv.state_dict(), "/content/drive/My Drive/NNProj/MNIST.pt")

In [0]:
def seed_init_fn(worker_id):
  np.random.seed(np.random.get_state()[1][0] + worker_id)
  return

In [44]:
use_cuda = torch.cuda.is_available()
print("CUDA? " + str(use_cuda))

torch.manual_seed(20294798)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                    ])),
    batch_size=100, shuffle=True, **kwargs, worker_init_fn = seed_init_fn)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, download=True, transform=transforms.Compose([
                        transforms.ToTensor(),
                    ])),
    batch_size=1, shuffle=True, **kwargs, worker_init_fn = seed_init_fn)

CUDA? True


In [0]:
def hype_pop(model, pop, testdata):
  model.eval()
  gen = []
  time_ = []
  
  for i, (data, target) in enumerate(testdata):
      if i >= 10000:
        break
      data, target = data.to(device), target.to(device)
      output = model(data)
      pred = output.argmax(dim=1, keepdim=True)

      if pred == target:
          start = time.time()
          adv_ex, success, G = GA.gen_attack(pop, data, target, 0.2, 0.15, 0.1, 5000, 2, model, GA.fitness, device)
          end = time.time()
          gen.append(G)
          time_.append(end-start)

  mean_gen = np.mean(gen)
  mean_time = np.mean(time_)
  return (mean_gen, mean_time)

In [0]:
def hype_mut(model, rho, testdata):
  model.eval()
  gen = []

  for i, (data, target) in enumerate(testdata):
      if i >= 10000:
        break
      data, target = data.to(device), target.to(device)
      output = model(data)
      pred = output.argmax(dim=1, keepdim=True)

      if pred == target:
          adv_ex, success, G = GA.gen_attack(10, data, target, 0.2, 0.15, rho, 5000, 2, model, GA.fitness, device)
          gen.append(G)

  mean_gen= np.mean(gen)
  return mean_gen

In [53]:
pretrained_model = "/content/drive/My Drive/NNProj/MNIST.pt"

device = torch.device("cuda" if (use_cuda) else "cpu")

model = Net().to(device)
model.load_state_dict(torch.load(pretrained_model, map_location='cpu'))
model.eval()

Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (dropout1): Dropout2d(p=0.25, inplace=False)
  (dropout2): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=9216, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [54]:
pops = [5, 10, 15, 20, 25]
MNIST_pop_gen = []
MNIST_pop_time = []
for pop in pops:
  print(pop)
  i, j = hype_pop(model, pop, test_loader)
  MNIST_pop_gen.append(i)
  MNIST_pop_time.append(j)

Streaming output truncated to the last 5000 lines.
Generation 4800
Plateau at Generation 4801
Generation 4900
Plateau at Generation 4901
Generation 100
Plateau at Generation 101
Generation 100
Generation 200
Generation 100
Plateau at Generation 101
Generation 200
Plateau at Generation 201
Generation 300
Plateau at Generation 301
Generation 400
Plateau at Generation 401
Generation 500
Plateau at Generation 501
Generation 600
Plateau at Generation 601
Generation 700
Plateau at Generation 701
Generation 800
Plateau at Generation 801
Generation 900
Plateau at Generation 901
Generation 1000
Plateau at Generation 1001
Generation 1100
Plateau at Generation 1101
Generation 1200
Plateau at Generation 1201
Generation 1300
Plateau at Generation 1301
Generation 1400
Plateau at Generation 1401
Generation 1500
Plateau at Generation 1501
Generation 1600
Plateau at Generation 1601
Generation 1700
Plateau at Generation 1701
Generation 1800
Plateau at Generation 1801
Generation 1900
Plateau at Generatio

In [0]:
import csv
with open('MNIST_pop.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(pops)
    writer.writerow(MNIST_pop_gen)
    writer.writerow(MNIST_pop_time)

In [49]:
prob = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3]
MNIST_rho_time = []
for rho in prob:
  print(rho)
  j = hype_mut(model, rho, test_loader)
  MNIST_rho_time.append(j)

0.05
Generation 100
Generation 200
Generation 300
Generation 400
Generation 500
Plateau at Generation 513
Generation 100
Generation 100
Generation 200
Generation 100
Plateau at Generation 184
Generation 200
Plateau at Generation 284
Generation 300
Plateau at Generation 384
Generation 400
Plateau at Generation 484
Generation 500
Generation 100
Generation 200
Generation 300
Generation 100
Generation 100
Generation 100
Generation 100
Generation 100
Generation 200
Generation 300
Plateau at Generation 354
Generation 400
Plateau at Generation 454
Generation 500
Plateau at Generation 554
Generation 600
Plateau at Generation 654
Generation 700
Plateau at Generation 754
Generation 800
Plateau at Generation 854
Generation 900
Plateau at Generation 954
Generation 1000
Plateau at Generation 1054
Generation 1100
Plateau at Generation 1154
Generation 1200
Plateau at Generation 1254
Generation 1300
Plateau at Generation 1354
Generation 1400
Plateau at Generation 1454
Generation 1500
Plateau at Genera

In [0]:
with open('MNIST_rho.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(prob)
    writer.writerow(MNIST_rho_time)